In [7]:
import pandas as pd
import lightgbm as lgb
import numpy as np
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm

from cate.dataset import Dataset

In [2]:
df = pd.read_csv("/workspace/data/uplift-modeling/criteo-uplift-v2.1.csv")

In [3]:
df.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,treatment,conversion,visit,exposure
0,12.616365,10.059654,8.976429,4.679882,10.280525,4.115453,0.294443,4.833815,3.955396,13.190056,5.300375,-0.168679,1,0,0,0
1,12.616365,10.059654,9.002689,4.679882,10.280525,4.115453,0.294443,4.833815,3.955396,13.190056,5.300375,-0.168679,1,0,0,0
2,12.616365,10.059654,8.964775,4.679882,10.280525,4.115453,0.294443,4.833815,3.955396,13.190056,5.300375,-0.168679,1,0,0,0
3,12.616365,10.059654,9.002801,4.679882,10.280525,4.115453,0.294443,4.833815,3.955396,13.190056,5.300375,-0.168679,1,0,0,0
4,12.616365,10.059654,9.037999,4.679882,10.280525,4.115453,0.294443,4.833815,3.955396,13.190056,5.300375,-0.168679,1,0,0,0


In [4]:
ds = Dataset(
    df, ["f0", "f1", "f2", "f3", "f4", "f5", "f6", "f7", "f8", "f9", "f10", "f11"], ["conversion"], ["treatment"]
)

In [ ]:
pred_df = pd.DataFrame(np.zeros(len(ds)))
skf = StratifiedKFold(5, shuffle=True, random_state=42)
for train_idx, valid_idx in tqdm(skf.split(np.zeros(len(ds)), ds.y)):
    train_X = ds.X.iloc[train_idx]
    train_y = ds.y.iloc[train_idx]
    train_w = ds.w.iloc[train_idx]
    valid_X = ds.X.iloc[valid_idx]
    valid_y = ds.y.iloc[valid_idx]
    valid_w = ds.w.iloc[valid_idx]

    model = lgb.LGBMClassifier(importance_type="gain")
    model.fit(train_X, train_y, eval_set=[(valid_X, valid_y)])
    pred_df.iloc[valid_idx] = model.predict_proba(train_X) # type: ignore

0it [00:00, ?it/s]